In [33]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import math


def scrape_shows(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.text,"html.parser")
    shows = soup.find_all("li")[2:35]
    
    shows_data = []
    for row in shows:
        shows_data.append(row.find_all("a")[0].text.strip())
    return shows_data

In [34]:
link = "https://www.ibdb.com"
showlist = scrape_shows(link)

In [35]:
showlist

['A Bronx Tale The Musical',
 'Aladdin',
 'Beautiful The Carole King Musical',
 'Cats',
 'Chicago',
 'Cirque du Soleil Paramour',
 'Dear Evan Hansen',
 'Falsettos',
 'Fiddler on the Roof',
 'Hamilton',
 'Heisenberg',
 'Holiday Inn, The New Irving Berlin Musical',
 'In Transit',
 'Jersey Boys',
 'Kinky Boots',
 'Les Liaisons Dangereuses',
 'Matilda The Musical',
 'Natasha, Pierre & The Great Comet of 1812',
 'Oh, Hello on Broadway',
 'On Your Feet!',
 'School of Rock – The Musical',
 'Something Rotten!',
 'The Book of Mormon',
 'The Cherry Orchard',
 'The Color Purple',
 'The Encounter',
 'The Front Page',
 'The Humans',
 'The Illusionists - Turn of the Century',
 'The Lion King',
 'The Phantom of the Opera',
 'Waitress',
 'Wicked']

In [77]:
import re
shows_links = {}
for show in showlist:
    name = re.sub('[^A-Za-z0-9]+', '', show).lower()
    if name[0:3] == "the":
        name = name[3:]+name[0:3]
        
    l = "https://www.allmusicals.com/{}/{}.htm".format(name[0],name)
    shows_links[show] = l

In [78]:
shows_links

{'A Bronx Tale The Musical': 'https://www.allmusicals.com/a/abronxtalethemusical.htm',
 'Aladdin': 'https://www.allmusicals.com/a/aladdin.htm',
 'Beautiful The Carole King Musical': 'https://www.allmusicals.com/b/beautifulthecarolekingmusical.htm',
 'Cats': 'https://www.allmusicals.com/c/cats.htm',
 'Chicago': 'https://www.allmusicals.com/c/chicago.htm',
 'Cirque du Soleil Paramour': 'https://www.allmusicals.com/c/cirquedusoleilparamour.htm',
 'Dear Evan Hansen': 'https://www.allmusicals.com/d/dearevanhansen.htm',
 'Falsettos': 'https://www.allmusicals.com/f/falsettos.htm',
 'Fiddler on the Roof': 'https://www.allmusicals.com/f/fiddlerontheroof.htm',
 'Hamilton': 'https://www.allmusicals.com/h/hamilton.htm',
 'Heisenberg': 'https://www.allmusicals.com/h/heisenberg.htm',
 'Holiday Inn, The New Irving Berlin Musical': 'https://www.allmusicals.com/h/holidayinnthenewirvingberlinmusical.htm',
 'In Transit': 'https://www.allmusicals.com/i/intransit.htm',
 'Jersey Boys': 'https://www.allmusic

In [79]:
shows_links["Beautiful The Carole King Musical"]='https://www.allmusicals.com/b/beautifulthecaroleking.htm'
shows_links["The Lion King"]='https://www.allmusicals.com/l/lionking.htm'
shows_links["Matilda The Musical"] = 'https://www.allmusicals.com/m/matilda.htm'
shows_links["Natasha, Pierre & The Great Comet of 1812"] = 'https://www.allmusicals.com/n/natashapierreandthegreatcometof1812.htm'
shows_links["School of Rock – The Musical"] = 'https://www.allmusicals.com/s/schoolofrock.htm'

In [153]:
def scrape_lyrics_links(show_name):
    show_link = shows_links[show_name]
    if show_link is None:
        print(show_name +" has no lyrics")
        return False
    r = requests.get(show_link)
    soup = BeautifulSoup(r.text,"html.parser")
    
    try:
        table = soup.find("section",class_="lyrics-list")
        lyrics = table.find_all("a")
    
        lyrics_data = []
        for row in lyrics:
            lyrics_link = row["href"]
            lyrics_data.append(lyrics_link)
        return lyrics_data
    
    except (AttributeError, IndexError):
        print(show_name+" has no lyrics")
        return False
    except ValueError:
        print(show_name+" has no lyrics")
        return False

    
def scrape_lyrics_each(lyrics_link):
    try:
        r = requests.get(lyrics_link)
        soup = BeautifulSoup(r.text,"html.parser")
        table = soup.find("div", class_ = "span8 lyrics-content-main-text")
        content = table.find_all("div")[3].get_text()
        return content
    except (AttributeError, IndexError):
        return False
    except (ValueError,NameError):
        return False


def scrape_show_lyrics(show_name):
    lyrics_links = scrape_lyrics_links(show_name)
    if lyrics_links != False:
        filename = "{}.txt".format(show_name)
        with open(filename, "w") as f:
            for link in lyrics_links:
                if scrape_lyrics_each(link) != False:
                    content = scrape_lyrics_each(link)
                    f.write(content)
        return show_name
    else:
        return False
    
def scrape_all_shows_lyrics(showlist):
    c = 0
    for showname in showlist:
        if scrape_lyrics_links(showname) != False:
            scrape_show_lyrics(showname)
            c += 1
    print("Done!")
    print(str(c) +" shows with valid lyrics")
    

In [154]:
scrape_all_shows_lyrics(showlist)

A Bronx Tale The Musical has no lyrics
Cirque du Soleil Paramour has no lyrics
Heisenberg has no lyrics
Holiday Inn, The New Irving Berlin Musical has no lyrics
In Transit has no lyrics
Les Liaisons Dangereuses has no lyrics
Oh, Hello on Broadway has no lyrics
The Cherry Orchard has no lyrics
The Encounter has no lyrics
The Front Page has no lyrics
The Humans has no lyrics
The Illusionists - Turn of the Century has no lyrics
Done!


TypeError: unsupported operand type(s) for +: 'int' and 'str'